In [1]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

import re
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [7]:
# function to remove stopwords
def remove_stopwords(sent):
    sent_new = " ".join([i for i in sent if i not in stop_words])
    return sent_new

In [8]:
# function to process sentence
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [9]:
# extract word embeddings from GLOVE dataset
word_embeddings = {}
f = open('glove.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

## Import Data

In [10]:
df = pd.read_csv("241_summarization.csv")


## Tokenize Sentences

In [14]:
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [15]:
sentences 

["I've got a question on the sample code for calculating robust and clustered SEs.",
 "When we're taking clustering into account (starting line ~100 of the example file), do we also need to fit a new model with the cluster as a covariate, or is alright to stick with the initial model defined on line ~81",
 'If we include an indicator for the clusters -- what are we going to estimate on that feature?',
 "Basically, like every other dummy variable that we use, we'll estimate the difference in outcomes between cluster -- which might be useful for reducing the variance.",
 "What would happen if we don't include it though?",
 "Good question -- if we don't include it, we won't estimate or account for any cluster level effects",
 'Yep yep; we won’t estimate any cluster level, mean differences.',
 'But, the estimate of the standard error isn’t predicated on having a measure of those cluster level differences.',
 'Would it be wrong to include cluster as a covariate?',
 "I was a little surprised

## Clean Text

In [26]:
s = " "
clean_text = s.join(sentences)
clean_text

"I've got a question on the sample code for calculating robust and clustered SEs. When we're taking clustering into account (starting line ~100 of the example file), do we also need to fit a new model with the cluster as a covariate, or is alright to stick with the initial model defined on line ~81 If we include an indicator for the clusters -- what are we going to estimate on that feature? Basically, like every other dummy variable that we use, we'll estimate the difference in outcomes between cluster -- which might be useful for reducing the variance. What would happen if we don't include it though? Good question -- if we don't include it, we won't estimate or account for any cluster level effects Yep yep; we won’t estimate any cluster level, mean differences. But, the estimate of the standard error isn’t predicated on having a measure of those cluster level differences. Would it be wrong to include cluster as a covariate? I was a little surprised it wasn't included in the example se

## Entity Recognition

In [28]:
text = nlp(clean_text)
items = [x.text for x in text.ents]
Counter(items).most_common(10)

[('beta_2', 2),
 ('Alex', 1),
 ('F', 1),
 ('`y ~ beta_0 + beta_1_woman +', 1),
 ('beta_3', 1)]

In [30]:
text.ents


(Alex, F, `y ~ beta_0 + beta_1_woman +, beta_2, beta_2, beta_3)

## Generate Summary

In [13]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [14]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
# Extract top 5 sentences as the summary
for i in range(5):
    print(ranked_sentences[i][1])

I just don't like how long it takes to actually GET that feedback.
office hours are really helpful and the TAs really seem like they understand the material.
the live session was just a repeat of the async material, and didn't feel like it was worth the time.
the TA didn't spend enough time on the topics we need for the problem set.
i liked the way he applied last week's material to projects he had worked on in his day job.
